In [ ]:
#  iGAMING PAGE ANALYSIS
# Checks if pages rank for iGaming terms in top 20

import pandas as pd
import requests
import base64
import time

# Configuration
DFS_EMAIL = "admin@wldm.io"
DFS_API_KEY = "cb54e37f6a4874eb"
DFS_BASE_URL = "https://api.dataforseo.com/v3"
LINKBUILDER_URL = "https://docs.google.com/spreadsheets/d/1gBEIThc5Lg3ZdkRMe7NQE8o371AlYt6b37rko-rdpsA/export?format=csv"

def get_dfs_headers():
    credentials = f"{DFS_EMAIL}:{DFS_API_KEY}"
    encoded_credentials = base64.b64encode(credentials.encode()).decode()
    return {"Authorization": f"Basic {encoded_credentials}", "Content-Type": "application/json"}

def get_page_keywords(domain):
    """Get top 20 ranking keywords for a domain"""
    endpoint = f"{DFS_BASE_URL}/dataforseo_labs/google/ranked_keywords/live"
    data = [{"target": domain, "location_code": 2840, "language_code": "en", "limit": 20}]

    try:
        response = requests.post(endpoint, json=data, headers=get_dfs_headers())
        if response.status_code == 200:
            results = response.json()
            keywords = []
            if 'tasks' in results and results['tasks']:
                task = results['tasks'][0]
                if 'result' in task and task['result']:
                    for item in task['result']:
                        if 'items' in item and item['items']:
                            for kw_item in item['items']:
                                keyword_data = kw_item.get('keyword_data', {})
                                keyword = keyword_data.get('keyword', '')
                                if keyword:
                                    keywords.append(keyword)
            return keywords
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []

# Load and filter domains to only iGaming sites
print("Loading iGaming domains...")
domains_df = pd.read_csv(LINKBUILDER_URL)

# Filter to only actual iGaming domains
igaming_domains = domains_df[
    domains_df['Domain'].str.contains('casino|poker|bet|gambl|slot', case=False, na=False)
]['Domain'].tolist()

print(f"Found {len(igaming_domains)} iGaming domains")
test_domains = igaming_domains[:10]  # Test first 10

print(f"Testing {len(test_domains)} iGaming domains for top 20 rankings...\n")

# Analyze each domain
results = []
for domain in test_domains:
    print(f"Checking: {domain}")

    keywords = get_page_keywords(domain)

    # Check for iGaming terms in top 20
    igaming_terms = ['blackjack', 'casino', 'poker', 'bet', 'gambl', 'slot']
    has_igaming = any(any(term in kw.lower() for term in igaming_terms) for kw in keywords)



    domain_data = domains_df[domains_df['Domain'] == domain].iloc[0]
    dr = domain_data['Domain Rating (DR)']
    traffic = domain_data['Traffic (Ah)']

    # Check if meets all criteria
    meets_criteria = dr >= 20 and traffic >= 1500 and has_igaming

    status = "QUALIFIED" if meets_criteria else "Not qualified"
    print(f"  DR: {dr}, Traffic: {traffic}, iGaming in top 20: {has_igaming} -> {status}")

    if has_igaming:
        igaming_kws = [kw for kw in keywords if any(term in kw.lower() for term in igaming_terms)]
        print(f"  iGaming keywords: {igaming_kws[:2]}")

    results.append({
        'Page_URL': f"https://{domain}",
        'Domain': domain,
        'DR': dr,
        'Traffic': traffic,
        'iGaming_In_Top20': has_igaming,
        'Status': 'QUALIFIED' if meets_criteria else 'NOT_QUALIFIED'
    })

    print()
    time.sleep(2)

# Final results
qualified = [r for r in results if r['Status'] == 'QUALIFIED']

print(f"Domains tested: {len(results)}")
print(f"Qualified pages: {len(qualified)}")
print(f"Success rate: {len(qualified)/len(results)*100:.0f}%")



if qualified:
    print(f"\nQUALIFIED PAGES:")
    for page in qualified:
        print(f"- {page['Page_URL']} (DR: {page['DR']}, Traffic: {page['Traffic']})")



    # Save results
    pd.DataFrame(results).to_csv('qualified_igaming_pages.csv', index=False)
    print(f"\nResults saved to: qualified_igaming_pages.csv")

Loading iGaming domains...
Found 96 iGaming domains
Testing 10 iGaming domains for top 20 rankings...

Checking: latestcasinosreviews.com
  DR: 17.0, Traffic: 0, iGaming in top 20: False -> Not qualified

Checking: casinopelitnetissa.com
  DR: 18.0, Traffic: 0, iGaming in top 20: True -> Not qualified
  iGaming keywords: ['scandibet']

Checking: netticasinokolikkopelit.com
  DR: 5.0, Traffic: 5, iGaming in top 20: True -> Not qualified
  iGaming keywords: ['huone casino']

Checking: netent-casinot.com
  DR: 6.0, Traffic: 0, iGaming in top 20: False -> Not qualified

Checking: parhaat-netticasinot.org
  DR: 2.0, Traffic: 0, iGaming in top 20: False -> Not qualified

Checking: netcasino.fi
  DR: 27.0, Traffic: 16, iGaming in top 20: True -> Not qualified
  iGaming keywords: ['paras netti casino']

Checking: netticasino-finland.com
  DR: 6.0, Traffic: 6, iGaming in top 20: False -> Not qualified

Checking: netticasino-bonus.com
  DR: 1.0, Traffic: 0, iGaming in top 20: False -> Not qualif